In [ ]:
import os
import numpy as np
from pathlib import Path
import librosa
from sklearn.model_selection import train_test_split
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_features, select_features
from tsfresh.feature_extraction import EfficientFCParameters
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
RAW_DATASET_PATH = os.path.join("..", "animal_sounds")
DATASET_AUDIO_PATH = os.path.join("..", "animal_sounds_clips")
SAMPLING_RATE = 16000

In [ ]:
# Get the list of audio file paths along with their corresponding labels

class_names = os.listdir(DATASET_AUDIO_PATH)
print("Our class names: {}".format(class_names,))

train_audio_paths = []
valid_audio_paths = []
train_labels = []
valid_labels = []
for label, name in enumerate(class_names):
    # print("Processing material {}".format(name,))
    dir_path = Path(DATASET_AUDIO_PATH) / name
    speaker_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.lower().endswith(".wav")
    ]
    label = label // 2 # coz every dir has a _valid copy
    if name.endswith("_valid"):
        valid_audio_paths += speaker_sample_paths
        valid_labels += [label] * len(speaker_sample_paths)
    else:
        train_audio_paths += speaker_sample_paths
        train_labels += [label] * len(speaker_sample_paths)
    # print(f"Loaded {len(speaker_sample_paths)} files from class {label}.")
    
print(
    "Found {} files belonging to {} classes.".format(len(train_audio_paths) + len(valid_audio_paths), len(class_names)//2)
)

In [ ]:
def path_to_audio(path):
    y, _ = librosa.load(path, sr=SAMPLING_RATE)
    return y

def paths_and_labels_to_dataset(audio_paths, labels):
    """Constructs a dataset of audios and labels."""
    audios = [path_to_audio(path) for path in audio_paths]
    return np.array(audios), np.array(labels)

def to_dataframe(dataset):
    column_values = [i + 1 for i in range(len(dataset[0]))]
    df = pd.DataFrame(data=dataset, columns=column_values)
    df["id"] = df.index
    df = df.melt(id_vars="id", var_name="time").sort_values(["id", "time"]).reset_index(drop=True)
    return df
        

In [ ]:
train_dataset, train_labels = paths_and_labels_to_dataset(train_audio_paths, train_labels)
test_dataset, test_labels = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)

In [ ]:
X_train = to_dataframe(train_dataset)
X_test = to_dataframe(test_dataset)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
train_to_take = 1_000_000

X_train_features = extract_features(X_train[X_train['id'] < train_to_take], column_id="id", column_sort="time", impute_function=impute, default_fc_parameters=EfficientFCParameters())
y_train = train_labels[:train_to_take]
X_test_features = extract_features(X_test, column_id="id", column_sort="time", impute_function=impute, default_fc_parameters=EfficientFCParameters())
y_test = test_labels


In [ ]:
X_train_features

In [ ]:
y_train

In [ ]:
relevant_features = set()

for label in np.unique(y_train):
    y_train_binary = y_train == label
    X_train_filtered = select_features(X_train_features, y_train_binary)
    print("Number of relevant features for class {}: {}/{}".format(label, X_train_filtered.shape[1], X_train_features.shape[1]))
    relevant_features = relevant_features.union(set(X_train_filtered.columns))

In [ ]:
X_train_filtered = X_train_features[list(relevant_features)]
X_test_filtered = X_test_features[list(relevant_features)]

In [ ]:
X_train_filtered.columns = [i for i in range(X_train_filtered.shape[1])]
X_test_filtered.columns = [i for i in range(X_test_filtered.shape[1])]

In [ ]:
clf = lgb.LGBMClassifier()
clf.fit(X_train_filtered, y_train)

In [ ]:
y_pred=clf.predict(X_test_filtered)

In [ ]:

accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy))

In [ ]:

cm = confusion_matrix(y_test, y_pred).astype(np.float64)
print(cm, cm.shape)
for i in range(cm.shape[0]):
  cm[i, :] /= np.sum(cm[i, :])
  
cm_matrix = pd.DataFrame(data=cm)
cm_matrix['types'] = np.array(['kidney','liver','muscle', 'ribs', 'skin'])

cm_matrix.set_index('types', inplace=True)
cm_matrix.columns = ['kidney','liver','muscle', 'ribs', 'skin']

plt.figure(figsize=(7, 5))
sns.heatmap(cm_matrix, annot=True, fmt=".3f", square=True, cbar=False, cmap="Blues", linewidths=3, vmin=0, vmax=1)
plt.xlabel("Predicted label", labelpad=16)
plt.ylabel("True label", labelpad=12)
plt.tick_params(axis='y', rotation=0)
plt.show()